## Fine Tune Language Model

Since our questions come from the Stack Exchange network, it's probably useful for us to fine-tune a model against all of the questions and answers on this site.

I've never done this before, so let's try to figure it out.

In [36]:
import os
import glob
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [70]:
# Find all directories containing Stack Exchange posts
directories = glob.glob('data/*stackexchange.com')
directories[:2]

['data/academia.meta.stackexchange.com', 'data/ai.stackexchange.com']

In [69]:
x = Path('data/academia.meta.stackexchange.com')
x.name

'academia.meta.stackexchange.com'

In [63]:
x.parent

PosixPath('data')

In [71]:
# Take each site and save it's raw text in single .txt file
def save_site_text_as_txt_file(directory:Path):
    
    output_name = x.parent / (directory.name + '.txt')
    with open(output_name, 'w') as out_file:
        
        
        # Open up the XML file for all posts
        full_path = directory/"Posts.xml"
        tree = ET.parse(full_path)
        root = tree.getroot()
        for child in root.getchildren():
            # Read HTML post with BeautifulSoup, strip tags
            soup = BeautifulSoup(child.attrib['Body'])
            text = soup.get_text()

            # write it to the file
            out_file.write(text)
            out_file.write('\n')
        

In [72]:
# Get .txt file for every site we've downloaded
for directory in directories:
    save_site_text_as_txt_file(Path(directory))